# 動作確認用のファイル

In [99]:
import pandas as pd
import math

In [2]:
tanka = pd.read_csv("kindai.csv",header=None)
waka = pd.read_csv("waka_half.csv",header=None)

In [3]:
print(tanka.info())
print(tanka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13963 entries, 0 to 13962
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       13963 non-null  object
dtypes: object(1)
memory usage: 109.2+ KB
None
                                     0
0  あああらき なげきやながき くろかみも ひとときにして あせやしぬべし
1  あああらき なげきやなんじ れはかりは てんもうごけと なくにもにたる
2  ああきみは いたましきまで ようちなる なさけにゑひぬ かなしからずや
3  ああきみは かたるにたらず よわきじん よわきこいをば もてあそぶかな
4  ああきみは すむむねもなき さびしさを かたるかなみだ かわきしめもて


In [4]:
print(waka.info())
print(waka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14292 entries, 0 to 14291
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       14292 non-null  object
dtypes: object(1)
memory usage: 111.8+ KB
None
                                     0
0  あかからは みるへきものを かりかねの いつこはかりに なきてゆくらむ
1  あかさりし おもかけはかり うつしても ちきりはさてや やまのゐのみつ
2  あかさりし きみかにほひの こひしさに うめのはなをそ けさはをりつる
3  あかさりし そてかとにほふ うめかかに おもひなくさむ あかつきのそら
4  あかさりし そてのわかれの なみたより かたみかほなる つきそみにそふ


In [145]:
def vectorize_df(df,n=3):
    list_df = df.apply(split_text,axis=1)
    tri_gram_df = list_df.apply(n_gram,n=n)
    vec_id_dic = calc_vector(tri_gram_df)
    out_df = convert_to_vector(tri_gram_df,vec_id_dic)
    return out_df

def split_text(text_record):
    return text_record.loc[0].split(" ")

def n_gram(text_list,n):
    ans = []
    for text in text_list:
        gram = []
        for i in range(len(text)-(n-1)):
            gram.append(text[i:i+n])
        ans.extend(gram)
    return ans

def calc_vector(gram_df):
    vector = {}
    vector["vector"] = {}
    vector["id"] = {}
    r2 = 0
    id = 0
    for line in gram_df:
        for word in line:
            if word in vector["vector"].keys():
                vector["vector"][word] += 1
            else:
                vector["vector"][word] = 1
                vector["id"][word] = id
            id += 1

    for key in vector["vector"].keys():
        r2 += vector["vector"][key]**2

    r = math.sqrt(r2)
    for key in vector["vector"].keys():
        vector["vector"][key] /= r

    return vector

def convert_to_vector(df,dic):
    out_vec_list = []
    out_id_list = []
    out_df = pd.DataFrame()
    for i in range(df.shape[0]):
        vec_list = []
        id_list = []
        word_list = df.loc[i]
        for word in word_list:
            vec_list.append(dic["vector"][word])
            id_list.append(dic["id"][word])
        out_vec_list.append(vec_list)
        out_id_list.append(id_list)
    out_df["vector"] = out_vec_list
    out_df["id"] = out_id_list
    return out_df


In [146]:
waka_vector_id = vectorize_df(waka,n=1)
tanka_vector_id = vectorize_df(tanka)

In [147]:
waka_path = "waka_vector.csv"
tanka_path = "tanka_vector.csv"

waka_vector_id.to_csv(waka_path,index=False)
tanka_vector_id.to_csv(tanka_path,index=False)

In [148]:
pd.read_csv(waka_path)

,vector,id
0,"[0.10288089429287038, 0.34179304836124225, 0.3...","[0, 1, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 13, ..."
1,"[0.10288089429287038, 0.34179304836124225, 0.1...","[0, 1, 33, 13, 35, 36, 9, 1, 39, 4, 1, 13, 43,..."
2,"[0.10288089429287038, 0.34179304836124225, 0.1...","[0, 1, 33, 13, 35, 8, 5, 1, 23, 71, 72, 10, 19..."
3,"[0.10288089429287038, 0.34179304836124225, 0.1...","[0, 1, 33, 13, 35, 85, 26, 1, 101, 23, 71, 104..."
4,"[0.10288089429287038, 0.34179304836124225, 0.1...","[0, 1, 33, 13, 35, 85, 26, 10, 132, 1, 134, 10..."
...,...,...
14287,"[0.09517362993400533, 0.17505010065828358, 0.0...","[11, 6, 85, 26, 23, 23, 71, 72, 4, 101, 56, 6,..."
14288,"[0.09517362993400533, 0.17505010065828358, 0.0...","[11, 6, 85, 26, 9, 17, 170, 85, 43, 18, 170, 1..."
14289,"[0.09517362993400533, 0.17505010065828358, 0.1...","[11, 6, 72, 101, 10, 85, 134, 24, 6, 1, 3, 23,..."
14290,"[0.09517362993400533, 0.17505010065828358, 0.1...","[11, 6, 72, 101, 9, 24, 8, 104, 6, 33, 101, 10..."
